# Follow up
In the previous notebook we took a micro sample and tried to find the best model and preprocessing based on it:
https://www.kaggle.com/thomasmeiner/naive-model-overview-using-automl-on-micro-sample 

Now we make use of our results and train a model finally.


In [ ]:
!pip install interface_meta --no-index --find-links=file:../input/e2eml-inc-dependencies/interface_meta-1.2.4-py2.py3-none-any.whl
!pip install astor --no-index --find-links=file:../input/e2eml-inc-dependencies/astor-0.8.1-py2.py3-none-any.whl
!pip install formulaic --no-index --find-links=file:../input/formulaic/formulaic-0.2.4-py3-none-any.whl
!pip install autograd --no-index --find-links=file:../input/e2eml-inc-dependencies/autograd-1.3-py3-none-any.whl
!pip install autograd-gamma --no-index --find-links=file:../input/e2eml-inc-dependencies/autograd-gamma-0.5.0/dist/autograd-gamma-0.5.0.tar
!pip install lifelines --no-index --find-links=file:../input/lifelines/lifelines-0.26.4-py3-none-any.whl
!pip install ngboost --no-index --find-links=file:../input/e2eml-inc-dependencies/ngboost-0.3.12-py3-none-any.whl
!pip install boostaroota --no-index --find-links=file:../input/e2eml-inc-dependencies/boostaroota-1.3-py2.py3-none-any.whl
!pip install matplotlib --no-index --find-links=file:../input/e2eml-inc-dependencies/matplotlib-3.1.3-cp38-cp38-manylinux1_x86_64.whl
!pip install catboost --no-index --find-links=file:../input/e2eml-inc-dependencies/catboost-0.21-cp38-none-manylinux1_x86_64.whl
!pip install pytorch_tabnet --no-index --find-links=file:../input/e2eml-inc-dependencies/pytorch_tabnet-3.1.1-py3-none-any.whl
!pip install shap --no-index --find-links=file:../input/e2eml-inc-dependencies/shap-0.39.0-cp35-cp35m-linux_armv6l.whl
!pip uninstall scikit-learn -y
!pip install scikit-learn --no-index --find-links=file:../input/sklearn-1-0/scikit_learn-1.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
!pip install e2eml --no-index --no-dependencies --find-links=file:../input/e2eml-inc-dependencies/e2eml-2.10.4-py3-none-any.whl

In [ ]:
import pandas as pd
import numpy as np
# load libraries
import sys
from e2eml.regression import regression_blueprints
from e2eml.full_processing import postprocessing
from e2eml.full_processing.postprocessing import save_to_production, load_for_production
from e2eml.timetravel import timetravel
from sklearn.model_selection import train_test_split
import gc

In [ ]:
target = "target"

# Automl using e2eml

We use e2eml. There are plenty of fantastic frameworks. Chose whatever you like.
Here we:
- load our prepared blueprint (see part 2 for the creation part)
- run the blueprint to get new predictions
- keep memory clean

Pretty elegant :-) 

In [ ]:
market_ml = load_for_production(file_path='../input/ubiquant-e2eml-lgbm/', file_name='market_ml_lgbm_instance.dat')

In [ ]:
import ubiquant
env = ubiquant.make_env()
# initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df["time_id"] = 0
    market_ml = load_for_production(file_path='../input/ubiquant-e2eml-lgbm/', file_name='market_ml_lgbm_instance.dat') # load Blueprint
    market_ml.ml_bp12_regressions_full_processing_lgbm(test_df) # predict
    sample_prediction_df['target'] =   market_ml.predicted_values['lgbm'] # attach predictions
    env.predict(sample_prediction_df)   # register your predictions
    del test_df
    del market_ml
    del sample_prediction_df
    _ = gc.collect()